In [15]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.animation as animation

In [16]:
# This function generates the positions of the hexagons for the hexagonal grid.
# Every even row has grid_width number of hexagons, and every odd row has
# grid_width - 1 hexagons. The odd rows are offset from the even rows by 
# half the width of a hexagon, because that's how geometry works. 
def gen_hex_grid(grid_width):
    grid_width = grid_width if grid_width % 2 == 1 else grid_width + 1
    start = -grid_width / 2.0 + 1 / 2
    end = grid_width / 2 - 1/2
    
    x_even = np.linspace(start, end, grid_width)
    x_odd = np.linspace(start + 1/2, end - 1/2, grid_width - 1)
    
    y_values = np.linspace(start, end, grid_width)
    
    x = []
    y = []
    
    for i in range(grid_width):
        curr_y = y_values[i]
        if (i % 2 == 0) :
            x += list(x_even)
            y += [curr_y for j in range(grid_width)]
        else:
            x += list(x_odd)
            y += [curr_y for j in range(grid_width - 1)]
        

    return x, y

In [17]:
# This function  determines which hexagons are adjacent to one another.
# The logic looks really gross just because it has to account for
# the edges of the grid, and because the logic is different for even and odd rows
# I think this structure does a few more comparisons than is necessary,
# but its a little more readable this way
def gen_hex_adj_dict(grid_width):
    grid_width = grid_width if grid_width % 2 == 1 else grid_width + 1 
    index = 0
    adj_dict = {}
    
    for y in range(grid_width):
        # whether the hexagon is on an even or odd row
        even_row = y % 2 == 0 

        # the number of hexagons in the current row
        x_count = grid_width if even_row else grid_width - 1

        # whether the hexagon is on a y-axis edge
        bottom_edge = y == 0
        top_edge = y == grid_width - 1
        
        for x in range(x_count):
            # whether the hexagon is on an x-axis edge
            left_edge = x == 0
            right_edge = x == x_count - 1 
            
            # the indices of a hexagon's spatially 
            # adjacent hexagons, if they exist
            right_hex = index + 1
            left_hex = index - 1
            upper_right = index + grid_width
            upper_left = index + grid_width - 1
            lower_right = index - grid_width
            lower_left = index - grid_width + 1
           
            adj_dict[index] = []
            
            # logic for adjacency to the right
            if not right_edge: 
                adj_dict[index].append(right_hex)
                
            # logic for adjacency to the left
            if not left_edge:
                adj_dict[index].append(left_hex)
                
            # logic for the y-axis adjacency of even rows
            if even_row:
                if not top_edge:
                    if not right_edge:
                        adj_dict[index].append(upper_right)
                    if not left_edge: 
                        adj_dict[index].append(upper_left)
                if not bottom_edge:
                    if not left_edge: 
                        adj_dict[index].append(lower_right)
                    if not right_edge:
                        adj_dict[index].append(lower_left)
                
            # logic for the y-axis adjacency of odd rows
            else: 
                if not top_edge:
                    adj_dict[index].append(upper_right)
                    adj_dict[index].append(upper_left)
                if not bottom_edge:
                    adj_dict[index].append(lower_right)
                    adj_dict[index].append(lower_left)
            
            index += 1
    return adj_dict
    

In [18]:
# these are the parameters for the animation!

# a couple notes:
# - grid_width must be odd, because of the way
#   hexbin treats the center of each hexagon
#
# - if delay is less than 3, back propogation will occur. Try it!
#
# - beat_rate = (grid_width + delay) will guarantee that 
#   each wave won't be interferred with. Changing it 
#   to small value (< 10) leads to some really interesting animations!
#
# - feel free to change the seed hexagon


grid_width = 17 # number of hexagons in each axis
delay = 3 # cooldown time of each hexagon
beat_rate = (grid_width + delay) # how often a new wave is initiated
seed = int(.5 * grid_width ** 2) # the starting hexagon

In [26]:
#Please run this code then click play on the player. Set mode to loop to see it run multiple times. 
#It functions in JupyterLab but not on github. Please contact 
#akazmi30@gatech.edu if you need to see a demo. Thank you! 



#define states 
ON = 300 #on is burning. I set the value high so that on the output it appears red! 
BURNT = 200 
GROWING= 100
OFF = 25 #off= can be ignited

FIRE_SPREAD_CHANCE = .9
BURNOUT_CHANCE = .4
REGROW_ALONE_CHANCE =.05
REGROW_NEIGHBOR_CHANCE = .2
FULLY_REGROW_CHANCE = .2

grid_width = 31 #Feel free to change this value (50 lets you see the cells more clearly)

adj_dict = gen_hex_adj_dict(grid_width)
x, y = gen_hex_grid(grid_width)
hex_count = len(x)

hex_state_global = [OFF for i in range(hex_count)]

#let's start a fire... (3 lines of fire essentially) 


#As before, this function is what takes in one state and gives us the next 
def transition(data):
    global hex_state_global# copy grid, similar to how we copied over the array in the 1D case
    #8 neighbors for calculation now! 
    hex_state = hex_state_global.copy()
        
    for i in range(hex_count):
        curr_state = hex_state_global[i]
        if curr_state == ON:
            for neighbor in adj_dict[i]:
                if hex_state[neighbor] == OFF and random.random() > FIRE_SPREAD_CHANCE:
                    hex_state[neighbor] = ON
            if random.random() > BURNOUT_CHANCE:
                hex_state[i] = BURNT
        elif curr_state == BURNT random.random() < REGROW_ALONE_CHANCE:
            hex_state[i] = GROWING
        elif curr_state == GROWING:
            for neighbor in adj_dict[i]:
                if hex_state[neighbor] == BURNT and random.random() > REGROW_NEIGHBOR_CHANCE:
                    hex_state[neighbor] = GROWING
            if random.random() < FULLY_REGROW_CHANCE:
                hex_state[i] = OFF
        
    # update data
    hex_state_global = hex_state
    return ax.hexbin(x, y, C = hex_state, gridsize = (grid_width - 1, int(grid_width / 2)), cmap = "summer")

# set up animation
hex_state_global[300] = ON
fig, ax = plt.subplots(figsize = (6,6))
mat = ax.hexbin(x, y, C = hex_state_global, gridsize = (grid_width - 1, int(grid_width / 2)), cmap = "summer")
ani = animation.FuncAnimation(fig, transition, interval=50,
                              save_count=50)

plt.close(fig)

#This code allows us to display the animation
#The animation has been tested to work in JupyterLab and Anaconda's Jupyter Notebooks

#When you first load the notebook you will need to run this cell (ctrl enter) to see the animation player. The image shown is the final state. 

#This will not display from within github.
from IPython.display import HTML
HTML(ani.to_jshtml())